## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [1]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28  --force-reinstall --upgrade --no-cache-dir -q 2>/dev/null

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 64.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 155.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 233.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 247.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 154.0 MB/s eta 0:00:00


**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [1]:
# For installing the libraries & downloading models from HF Hub
!pip install huggingface_hub==0.23.2 pandas==1.5.3 tiktoken==0.6.0 pymupdf==1.25.1 langchain==0.1.1 langchain-community==0.0.13 chromadb==0.4.22 sentence-transformers==2.3.1 numpy==1.25.2 -q

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [2]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Question Answering using LLM

#### Downloading and Loading the model

In [3]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

* Using the "MISTRAL" model for this project

In [4]:
model_path = hf_hub_download(
    repo_id= model_name_or_path,
    filename= model_basename
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

In [5]:
llm = Llama(
    model_path=model_path,
    n_ctx=2300,
    n_gpu_layers=38,
    n_batch=512
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


Loads a LLaMA-based model optimized for inference with a balance between performance and memory usage

#### Response

In [6]:
def response(query,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    model_output = llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

This function is a convenient wrapper for generating text using the llm model, with adjustable sampling parameters for fine-tuning output randomness and diversity

In [7]:
response("What treatment options are available for managing hypertension?")

'\n\nHypertension, or high blood pressure, is a common condition that can increase the risk of various health problems such as heart disease, stroke, and kidney damage. The good news is that there are several effective treatment options available to help manage hypertension and reduce the risk of complications. Here are some of the most commonly used treatments:\n\n1. Lifestyle modifications: Making lifestyle changes is often the first line of defense against hypertension. This may include eating a healthy diet rich in fruits, vegetables, whole grains, and lean proteins; limiting sodium intake; getting regular physical activity'

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [8]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
response(user_input)

Llama.generate: prefix-match hit


'\n\nSepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are general steps for managing sepsis in a critical care unit:\n\n1. Early recognition: Recognize the signs and symptoms of sepsis early and initiate treatment as soon as possible. Sepsis can present with various clinical features, including fever or hypothermia, tachycardia or bradycardia, altered mental status, respiratory distress, and lactic acidosis.\n2. ABCs'

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [9]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response(user_input)

Llama.generate: prefix-match hit


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small pouch-like structure that extends from the large intestine. The symptoms of appendicitis can vary from person to person, but some common signs include:\n\n1. Abdominal pain: The pain is typically located in the lower right side of the abdomen and may be constant or come and go. It may start as a mild discomfort that worsens over time.\n2. Loss of appetite: People with appendicitis often lose their appetite due to abdominal pain and nausea'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [10]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response(user_input)

Llama.generate: prefix-match hit


"\n\nSudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles. It can result in round or oval bald patches on the scalp, but it can also occur on other parts of the body such as the beard area, eyebrows, and eyelashes.\n\nThe exact cause of alopecia areata is not known, but it's believed to be related to a problem with the immune system. Some possible triggers for this condition include stress, genetics, viral infections, and certain medications."

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [11]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response(user_input)

Llama.generate: prefix-match hit


'\n\nA person who has sustained a physical injury to the brain tissue may require various treatments depending on the severity and location of the injury. Here are some common treatments that may be recommended:\n\n1. Emergency care: In case of a traumatic brain injury (TBI), it is essential to seek emergency medical attention as soon as possible. The primary goal of emergency care is to prevent further damage to the brain, stabilize vital signs, and manage any life-threatening conditions.\n2. Medications: Depending on the symptoms, healthcare professionals may prescribe medications to manage various conditions associated with a'

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [12]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response(user_input)

Llama.generate: prefix-match hit


"\n\nFirst and foremost, if you suspect that someone has fractured their leg while hiking, it's essential to ensure their safety and prevent further injury. Here are some necessary precautions:\n\n1. Keep the person calm and still: Encourage them to remain as still as possible to minimize pain and prevent worsening the injury.\n2. Assess the situation: Check for any signs of shock, such as pale skin, rapid heartbeat, or shallow breathing. If you notice these symptoms, seek medical help immediately.\n3. Immobilize the leg: Use a splint, sl"

## Question Answering using LLM with Prompt Engineering

In [13]:
system_prompt = "You are a helpful, accurate, and responsible medical assistant."

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [14]:
user_input = system_prompt+"\n"+ "What is the protocol for managing sepsis in a critical care unit?"
response(user_input)

Llama.generate: prefix-match hit


'\nSepsis is a life-threatening condition that requires prompt recognition and intervention. In a critical care unit, the following protocol should be followed for managing sepsis:\n1. Early Recognition: Identify sepsis early by recognizing signs and symptoms such as fever or hypothermia, tachycardia or bradycardia, respiratory distress, altered mental status, and lactic acidosis. Use the Sequential Organ Failure Assessment (SOFA) score to assess organ dysfunction.\n2. Fluid Resuscitation: Administer intravenous flu'

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [15]:
user_input = system_prompt+"\n"+ "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response(user_input)

Llama.generate: prefix-match hit


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small pouch that extends from the large intestine. The following are common symptoms of appendicitis:\n\n1. Abdominal pain: The pain typically starts around the navel area and then shifts to the lower right side of the abdomen. It may be constant or come and go, and it can range from mild to severe.\n2. Loss of appetite: People with appendicitis often lose their appetite due to abdominal pain and discomfort.\n3. Nausea and vomiting:'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [16]:
user_input = system_prompt+"\n"+ "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response(user_input)

Llama.generate: prefix-match hit


"\n\nSudden patchy hair loss, also known as alopecia areata, is an autoimmune condition that causes hair loss in small, round patches on the scalp or other areas of the body. The exact cause of alopecia areata is not known, but it's believed to be related to a problem with the immune system.\n\nThere are several treatments and solutions for addressing sudden patchy hair loss:\n\n1. Corticosteroids: These medications can help reduce inflammation and suppress the immune system response that causes hair loss. They can be applied topically or taken"

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [17]:
user_input = system_prompt+"\n"+ "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response(user_input)

Llama.generate: prefix-match hit


"\n\nI'm glad you asked about this important topic! When someone sustains a physical injury to the brain, also known as a traumatic brain injury (TBI), the recommended treatments can vary depending on the severity and specific location of the injury. Here are some common treatments that may be recommended:\n\n1. Emergency care: If the injury is severe, the first priority is to ensure the person receives appropriate emergency care. This may include measures to maintain breathing and circulation, control bleeding, and prevent further injury.\n2. Medications: Depending on the symptoms, medications may be prescribed to manage"

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [18]:
user_input = system_prompt+"\n"+ "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response(user_input)

Llama.generate: prefix-match hit


"\n\nFirst and foremost, it's important to ensure the safety and stability of the injured person. If possible, help them find a comfortable position to rest and reduce any pain or discomfort. Do not attempt to move them unless there is a risk of further injury or if they are in an unsafe location.\n\nOnce the person is stable, assess the extent of the injury by checking for signs of swelling, bruising, deformity, or open wounds. If you suspect a fracture, do not try to realign the bone as this could cause more harm. Instead, immobilize the leg using a"

## Data Preparation for RAG

In [19]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

### Loading the Data

In [20]:
pdf_path='/content/medical_diagnosis_manual.pdf'

In [22]:
pdf_loader = PyMuPDFLoader(pdf_path)

In [24]:
manual = pdf_loader.load()

### Data Overview

#### Checking the first 5 pages

In [25]:
for i in range(5):
    print(f"Page Number : {i+1}",end="\n")
    print(manual[i].page_content,end="\n")

Page Number : 1
govindasamy1017@gmail.com
NALHFPJWZB
t for personal use by govindasamy1017@
shing the contents in part or full is liable 

Page Number : 2
govindasamy1017@gmail.com
NALHFPJWZB
This file is meant for personal use by govindasamy1017@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

Page Number : 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ..............................................................................................................................................................................

* First 5 pages contents are listed successfully

#### Checking the number of pages

In [26]:
len(manual)

4114

No of pages are 4114

### Data Chunking

In [27]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap= 20
)

In [28]:
document_chunks = pdf_loader.load_and_split(text_splitter)

In [29]:
len(document_chunks)

8492

* After Data Chunking, the no of pages are 8492

In [30]:
document_chunks[0].page_content

'govindasamy1017@gmail.com\nNALHFPJWZB\nt for personal use by govindasamy1017@\nshing the contents in part or full is liable'

In [31]:
document_chunks[1].page_content

'govindasamy1017@gmail.com\nNALHFPJWZB\nThis file is meant for personal use by govindasamy1017@gmail.com only.\nSharing or publishing the contents in part or full is liable for legal action.'

In [32]:
document_chunks[2].page_content

'Table of Contents\n1\nFront    ................................................................................................................................................................................................................\n1\nCover    .......................................................................................................................................................................................................\n2\nFront Matter    ...........................................................................................................................................................................................\n53\n1 - Nutritional Disorders    ...............................................................................................................................................................\n53\nChapter 1. Nutrition: General Considerations    ...........................................................................................

In [33]:
document_chunks[3].page_content

"Chapter 24. Testing for Hepatic & Biliary Disorders    ......................................................................................................\n305\nChapter 25. Drugs & the Liver    ................................................................................................................................................\n308\nChapter 26. Alcoholic Liver Disease    ....................................................................................................................................\n314\nChapter 27. Fibrosis & Cirrhosis    ............................................................................................................................................\n322\nChapter 28. Hepatitis    ..................................................................................................................................................................\n333\nChapter 29. Vascular Disorders of the Liver    .................................................

As anticipated, there is some overlap:

* The phrase "to cultivate the experts-leading-experts model even within" is present in both segments.
* By increasing the chunk_overlap, the portion of the sentence that overlaps will grow accordingly.

### Embedding

In [34]:
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [35]:
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)

In [36]:
print("Dimension of the embedding vector ",len(embedding_1))
len(embedding_1)==len(embedding_2)

Dimension of the embedding vector  1024


True

* The embedding model provides a fixed-length vector for any number of chunks.
* This is necessary because we want to compare them for similarity.

In [37]:
embedding_1,embedding_2

([-0.031383316963911057,
  -0.011861815117299557,
  0.012422462925314903,
  0.0006565620424225926,
  -0.011360720731317997,
  -0.04826026409864426,
  0.0018251980654895306,
  0.029281344264745712,
  0.015422516502439976,
  0.017466522753238678,
  0.020778194069862366,
  -0.008075699210166931,
  0.013904563151299953,
  -0.03333742171525955,
  0.005161023698747158,
  -0.0025329708587378263,
  -0.005649976897984743,
  -0.03797079995274544,
  0.001188245601952076,
  0.008642245084047318,
  0.02036452107131481,
  0.015499277040362358,
  -0.06201332062482834,
  -0.02442634478211403,
  -0.002562983427196741,
  0.022926704958081245,
  0.02807503007352352,
  -0.0010970593430101871,
  0.05005151405930519,
  0.04373539239168167,
  -0.036521390080451965,
  -0.007737808860838413,
  0.04630522429943085,
  -0.05793033167719841,
  -0.020888684317469597,
  0.004818372428417206,
  0.03976595029234886,
  -0.010762210935354233,
  -0.013658038340508938,
  -0.02654552087187767,
  9.236309415427968e-05,
  -0

### Vector Database

In [38]:
out_dir = 'medical_db'

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

In [39]:
vectorstore = Chroma.from_documents(
    document_chunks,
    embedding_model,
    persist_directory=out_dir
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [40]:
vectorstore = Chroma(persist_directory=out_dir,embedding_function=embedding_model)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [41]:
vectorstore.embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='thenlper/gte-large', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

In [43]:
vectorstore.similarity_search("medical cancer",k=3)

[Document(page_content='Chapter 124. Principles of Cancer Therapy\nIntroduction\nCuring cancer requires eliminating all cancer cells. The major modalities of therapy are\n• Surgery and radiation therapy (for local and local-regional disease)\n• Chemotherapy (for systemic disease) Other important methods include\n• Hormonal therapy (for selected cancers, eg, prostate, breast, endometrium)\n• Immunotherapy (monoclonal antibodies, interferons, and other biologic response modifiers and tumor\nvaccines—see p. 1059)\n• Differentiating agents such as retinoids\n• Targeted agents that exploit the growing knowledge of cellular and molecular biology\nOverall treatment should be coordinated among a radiation oncologist, surgeon, and medical oncologist,\nwhere appropriate. Choice of modalities constantly evolves, and numerous controlled research trials\ncontinue. When available and appropriate, clinical trial participation should be considered and discussed\nwith patients.\nVarious terms are used 

* From the retrieved chunks, we observe that all the chunks are related to the key terms [ 'medical', 'cancer' ].

### Retriever

In [44]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 2}
)

In [45]:
rel_docs = retriever.get_relevant_documents("Explain about Chloramphenicol?")
rel_docs

[Document(page_content='Because of bone marrow toxicity, the availability of alternative antibiotics, and the emergence of\nresistance, chloramphenicol is no longer a drug of choice for any infection, except for\n• Serious infections due to a few multidrug-resistant bacteria that remain susceptible to this antibiotic\nHowever, when chloramphenicol has been used to treat meningitis caused by relatively penicillin-resistant\npneumococci, outcomes have been discouraging, probably because chloramphenicol has poor\nbactericidal activity against these strains.\nContraindications\nChloramphenicol is contraindicated if another drug can be used instead.\nThe Merck Manual of Diagnosis & Therapy, 19th Edition\nChapter 132. Bacteria & Antibacterial Drugs\n1326\ngovindasamy1017@gmail.com\nNALHFPJWZB\nThis file is meant for personal use by govindasamy1017@gmail.com only.\nSharing or publishing the contents in part or full is liable for legal action.', metadata={'author': '', 'creationDate': 'D:20120

* We can observe that the two relevant chunks contain the answer to the query.
* If we increase the k value, there is a chance that we might find the answer in even more chunks.
* This is a hyperparameter that we need to tune to get the best context.

In [46]:
# Combine the retrieved docs into a single context string
context = "\n\n".join([doc.page_content for doc in rel_docs])

# Create the prompt
prompt = f"Using the following context, answer the question:\n\n{context}\n\nQuestion: Explain about Chloramphenicol?"


In [47]:
model_output = llm(
      prompt,
      max_tokens=256,
      temperature=0.1
    )

Llama.generate: prefix-match hit


In [48]:
model_output['choices'][0]['text']

' What are its indications and contraindications?\n\nChloramphenicol is an antibiotic that inhibits bacterial protein synthesis by binding to the 50S subunit of the ribosome. It has a wide spectrum of activity against both gram-positive and gram-negative cocci, bacilli (including anaerobes), rickettsia, Mycoplasma, Chlamydia, and Chlamydophila spp. However, due to bone marrow toxicity, the availability of alternative antibiotics, and the emergence of resistance, chloramphenicol is no longer a drug of choice for most infections.\n\nThe only exceptions are serious infections caused by a few multidrug-resistant bacteria that remain susceptible to this antibiotic. For instance, chloramphenicol has been used to treat meningitis caused by relatively penicillin-resistant pneumococci, but the outcomes have been discouraging due to its poor bactericidal activity against these strains.\n\nChloramphenicol is contraindicated if another drug can be used instead. It is primarily bacteriostatic and i

### System and User Prompt Template

In [49]:
# 1. System message - defines the assistant’s role
qna_system_message = "You are a knowledgeable medical assistant. Use the provided context to give accurate, concise, and factual answers to medical questions."

# 2. User message template - includes context + question placeholders
qna_user_message_template = "Context:\n{context}\n\nQuestion: {question}\nAnswer:"

### Response Function

In [50]:
def generate_rag_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [51]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


'In a critical care unit, sepsis should be suspected based on clinical signs and symptoms, such as fever, tachycardia, hypotension, and respiratory distress. Empirical antibiotic therapy should be initiated immediately after taking specimens for Gram stain and culture. A common regimen includes gentamicin or tobramycin plus a 3rd-generation cephalosporin (cefotaxime or ceftriaxone), or ceftazidime plus a fluoroquinolone (ciprofloxacin). Vancomycin should be'

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [52]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


"The common symptoms for appendicitis include abdominal pain, particularly in the lower right side, loss of appetite, and abdominal tenderness. Appendicitis cannot be cured with medicine alone; instead, a surgical procedure called an appendectomy is required to remove the inflamed appendix. This can be done through open surgery or laparoscopically. Antibiotics are given intravenously before the surgery to help prevent infection spread. If the appendix has ruptured, antibiotics should be continued until the patient's temperature and white blood cell count return to normal. In cases where"

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [53]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


'Sudden patchy hair loss, also known as alopecia areata, is an autoimmune disorder that affects genetically susceptible individuals exposed to unclear environmental triggers. The most common treatments for alopecia areata include topical, intralesional, or systemic corticosteroids, topical minoxidil, topical anthralin, topical immunotherapy (diphencyprone or squaric acid dibutylester), or psoralen plus ultraviolet A (PUVA). The efficacy is usually evident within 6 to'

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [54]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


'The initial treatment for a traumatic brain injury (TBI) involves ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure. Surgery may be needed for patients with more severe injuries to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas. In the first few days after the injury, maintaining adequate brain perfusion and oxygenation and preventing complications of altered sensorium are important. Subsequently, many patients require rehabilitation. There is no specific treatment for TBI,'

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [55]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


'A fractured leg requires prompt medical attention. The first step is to immobilize the injured limb using an elastic bandage or a splint to prevent further injury and facilitate healing. The injured limb should be elevated above heart level for the first 2 days to minimize swelling and help drain edema fluid. After 48 hours, periodic application of warmth, such as a heating pad, for 15 to 20 minutes may relieve pain and speed up healing.\n\nIf the fracture requires weeks of immobilization, a cast is used instead of a splint. Patients with'

### Fine-tuning

Query 1: What is the protocol for managing sepsis in a critical care unit?

In [56]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input,temperature=0.5)

Llama.generate: prefix-match hit


'The management of sepsis in a critical care unit involves prompt empiric antibiotic therapy, source control if possible, and normalization of blood glucose levels. Antibiotics such as gentamicin or tobramycin with a third-generation cephalosporin (cefotaxime or ceftriaxone), or ceftazidime plus a fluoroquinolone (ciprofloxacin), are commonly used for septic shock of unknown cause. Vancomycin should be added if resistant staphylococci or enterococ'

Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [57]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
generate_rag_response(user_input,temperature=0.5)

Llama.generate: prefix-match hit


"The common symptoms of appendicitis include abdominal pain, usually localized in the lower right quadrant, anorexia, and abdominal tenderness. Appendicitis cannot be cured with medicine alone; instead, it requires surgical removal of the appendix through open or laparoscopic appendectomy. The surgery should precede antibiotics to prevent perforation and reduce morbidity and mortality. If the appendix is perforated, antibiotics should be continued until the patient's temperature and white blood cell count return to normal. In cases where the appendix cannot"

Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [58]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
generate_rag_response(user_input,temperature=0.5)

Llama.generate: prefix-match hit


"Sudden patchy hair loss, also known as alopecia areata, is an autoimmune disorder where the body's immune system attacks hair follicles. The most common treatments for alopecia areata include topical or intralesional corticosteroids, topical minoxidil, topical anthralin, topical immunotherapy (diphencyprone or squaric acid dibutylester), or psoralen plus ultraviolet A (PUVA). These treatments aim to stimulate hair growth and usually show effectiveness within 6-8 months."

Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [59]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_rag_response(user_input,temperature=0.5)

Llama.generate: prefix-match hit


'The initial treatment for traumatic brain injury (TBI) involves ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure. Surgery may be needed in more severe cases to place monitors to track and treat intracranial pressure or remove intracranial hematomas. In the first few days after the injury, maintaining adequate brain perfusion and oxygenation and preventing complications are important. Subsequently, many patients require rehabilitation. There is no specific treatment for TBI, but supportive care includes preventing systemic complications due to immobilization, providing good nutrition, and preventing'

Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [60]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
generate_rag_response(user_input,temperature=0.5)

Llama.generate: prefix-match hit


'A person with a fractured leg should first receive appropriate medical attention. The injured limb should be immobilized using either a cast or a splint to prevent further injury and facilitate healing. Layers 1, 3, and 4 of the bandaging should consist of non-elastic materials like gauze, while layers 2 and 4 are elastic bandages. The injured limb should be elevated above the heart for the first two days to minimize swelling and help drain edema fluid.\n\nAfter 48 hours, applying warmth for 15 to 20 minutes using a heating'

## Output Evaluation

Let us now use the LLM-as-a-judge method to check the quality of the RAG system on two parameters - retrieval and generation. We illustrate this evaluation based on the answeres generated to the question from the previous section.

- We are using the same Mistral model for evaluation, so basically here the llm is rating itself on how well he has performed in the task.

In [61]:
groundedness_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
The answer should be derived only from the information presented in the context

Instructions:
1. First write down the steps that are needed to evaluate the answer as per the metric.
2. Give a step-by-step explanation if the answer adheres to the metric considering the question and context as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the answer using the evaluaton criteria and assign a score.
"""

In [62]:
relevance_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
Relevance measures how well the answer addresses the main aspects of the question, based on the context.
Consider whether all and only the important aspects are contained in the answer when evaluating relevance.

Instructions:
1. First write down the steps that are needed to evaluate the context as per the metric.
2. Give a step-by-step explanation if the context adheres to the metric considering the question as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the context using the evaluaton criteria and assign a score.
"""

In [63]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [64]:
def generate_ground_relevance_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = llm(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    response_2 = llm(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [65]:
ground,rel = generate_ground_relevance_response(user_input="What is the protocol for managing sepsis in a critical care unit?",max_tokens=370)

print(ground,end="\n\n")
print(rel)


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify if the answer is derived only from the context provided.
2. Check if each step mentioned in the answer can be traced back to specific information in the context.

Explanation:
The AI generated answer adheres to the metric as it mentions steps for managing sepsis in a critical care unit based on the information provided in the context. The context discusses various aspects of critical care medicine, including patient monitoring, testing, and supportive care. The answer references these sections and specifically mentions the protocols for investigating alarms (mentioned in the context), the importance of prompt empiric therapy (discussed under "Point-of-care testing" and "Patient Monitoring and Testing"), and the need for source control and normalization of blood glucose (discussed under "Blood Tests").

Evaluation:
The metric is followed completely.

Rating:
Based on the evaluation criteria, I would rate the answer a 5.

 Steps to evaluate the 

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [66]:
ground,rel = generate_ground_relevance_response(user_input="What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",max_tokens=370)

print(ground,end="\n\n")
print(rel)


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the key information related to the question in the context.
2. Determine if the AI generated answer includes only the information from the context.
3. Check if the symptoms mentioned in the answer are consistent with those given in the context.
4. Verify if the treatment options and their sequence in the answer align with those stated in the context.
5. Ensure that no additional or irrelevant information is included in the answer.

Explanation:
The AI generated answer adheres to the metric as it only includes the information derived from the context. The symptoms mentioned (abdominal pain, anorexia, and abdominal tenderness) are consistent with those given in the context for appendicitis. The treatment options (surgical removal, IV antibiotics, open or laparoscopic appendectomy, etc.) and their sequence align with those stated in the context. There is no additional or irrelevant information included in the answer.

Evaluation:
The metric is fo

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [67]:
ground,rel = generate_ground_relevance_response(user_input="What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",max_tokens=370)

print(ground,end="\n\n")
print(rel)


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the main question and the key information in the context related to effective treatments and causes of sudden patchy hair loss.
2. Check if the AI generated answer only uses the information from the context to formulate its response.
3. Compare the details provided by the AI with the key points mentioned in the context to ensure consistency.

Explanation:
The AI generated answer adheres to the metric as it solely derives the information from the context provided without introducing any external or irrelevant data. The answer mentions various treatments for alopecia areata and their effectiveness, which is directly taken from the context. Additionally, it also explains that hormonal modulators and surgical options can be considered in severe cases, which is also mentioned in the context.

Evaluation:
The metric is followed completely as the answer is derived solely from the information presented in the context.

Rating:
Based on the evaluation 

### Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [68]:
ground,rel = generate_ground_relevance_response(user_input="What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",max_tokens=370)

print(ground,end="\n\n")
print(rel)


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the key information in the context related to treatments for traumatic brain injury (TBI).
2. Compare the identified information with the AI generated answer to determine if the answer is derived only from the context.

Explanation:
The context provides detailed information about the initial treatment and subsequent care for a person with TBI, including supportive care measures. The AI generated answer includes all of this information and does not introduce any new or irrelevant details. Therefore, the answer is derived solely from the context.

Evaluation:
The metric is followed completely.

Rating:
Based on the evaluation criteria, I would rate the answer a 5 for following the metric completely.

 Steps to evaluate the context as per the relevance metric:
1. Identify the main aspects of the question: The question asks about treatments for a person with a traumatic brain injury (TBI), focusing on both initial treatment and subsequent care.
2.

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [69]:
ground,rel = generate_ground_relevance_response(user_input="What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?",max_tokens=370)

print(ground,end="\n\n")
print(rel)


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the information in the context related to precautions and treatment steps for a fractured leg.
2. Compare the information in the context with the answer to ensure that the answer is derived only from the context.
3. Evaluate the extent to which the answer adheres to the metric by checking if all the information in the answer can be found in the context.

Explanation:
The AI generated answer follows the metric to a good extent as it includes all the necessary precautions and treatment steps for a person who has fractured their leg, which are derived directly from the context. However, some additional information is provided in the answer that is not explicitly stated in the context, such as "Some rapidly healing injuries may be best treated with early mobilization to minimize contractures and muscle atrophy." While this information is related to the topic of fracture treatment, it goes beyond what is specifically mentioned in the context.

Rati

## Actionable Insights and Business Recommendations

1. Enable Rapid Information Retrieval
    * Use the LLM to allow medical professionals, students, and researchers to quickly retrieve accurate, domain-specific information.

2.  Maintain Context Through Chunking
    * Apply chunk_overlap to preserve coherence when critical context spans multiple chunks.

3. Tune Token Limits by Query Complexity
    * Use higher max_tokens for complex queries to generate comprehensive responses.
    * Even with large token limits, simple queries should remain concise through prompt design and low temperature settings.

4. Optimize Prompt Design & Temperature
    * Refine prompt phrasing and temperature to control response length, creativity, and factual precision.

5. Continuously Adjust RAG Parameters
    * Adapt retrieval-augmented generation (RAG) parameters to fit the needs of specific use cases, balancing depth and speed.

6. Prioritize Output Quality
    * Evaluate responses for groundedness (alignment with source data) and relevance (query-specific usefulness).

7. Establish a Feedback Loop
    * Collect user feedback to iteratively fine-tune prompts, parameters, and retrieval settings, ensuring optimal performance for both simple and complex medical queries.

<font size=6 color='blue'>Power Ahead</font>
___